# Day 03 — Tool routing (structured outputs)

Today I practice **tool routing**: asking the model to choose a tool and return structured JSON.
I’ll add more step-by-step commentary so it reads like a tutorial.


## What we are building
We will: 
- Define a tiny tool catalog.
- Ask the model to pick a tool with JSON output.
- Parse that JSON and execute the chosen tool locally.

This is the core pattern behind many tool-using agents.


## Imports + helpers
We load the router prompt from disk and use a small `render` helper
to inject the user request.


In [ ]:
import json
from pathlib import Path

from openai import OpenAI

client = OpenAI()
PROMPTS_DIR = Path("prompts")

def load_prompt(name: str) -> str:
    return (PROMPTS_DIR / name).read_text()

def render(template: str, **kwargs: str) -> str:
    text = template
    for key, value in kwargs.items():
        text = text.replace(f"{{{key}}}", value)
    return text

def call_openai(prompt: str, model: str = "gpt-4o-mini") -> str:
    response = client.responses.create(
        model=model,
        input=prompt,
    )
    return response.output_text


## Define local tools
Each tool is just a Python function. The router decides which one to call
and provides arguments in JSON.


In [ ]:
def summarize_text(text: str) -> str:
    return "Summary: " + text.split(".")[0] + "."

def estimate_budget(items: list[str]) -> str:
    return f"Estimated budget for {len(items)} items: $50"

def create_study_plan(topic: str) -> str:
    return f"3-step study plan for {topic}: 1) Basics 2) Practice 3) Review"


## Ask the model to route the request
The routing prompt lists available tools and demands JSON output.
We can then parse the JSON and call the tool function locally.


In [ ]:
request = "Create a study plan for learning linear regression in two days."

router_prompt = render(
    load_prompt("router.txt"),
    request=request,
)

tool_call = call_openai(router_prompt)
tool_call


## Parse and execute
At this point we expect JSON like:
`{"tool": "create_study_plan", "arguments": {"topic": "linear regression"}}`
We parse it and route to the Python function.


In [ ]:
payload = json.loads(tool_call)
tool_name = payload["tool"]
args = payload["arguments"]

tool_map = {
    "summarize_text": summarize_text,
    "estimate_budget": estimate_budget,
    "create_study_plan": create_study_plan,
}

result = tool_map[tool_name](**args)
result
